In [1]:
import pandas as pd
import json
import string

In [2]:
all_files_za = pd.read_csv("Classification/all_files_ZAv1.csv")

In [3]:
def getanartistsjson(all_files_za, artist_name):

    artist_df = all_files_za[all_files_za["main_artist"] == artist_name]
    
    num_of_streams = artist_df['streams'].sum()
    min_date = artist_df['End Date'].min()
    max_date = artist_df['End Date'].max()

    unique_songs = artist_df['track_name'].unique().tolist()
    num_unique_songs = len(unique_songs)
    
    source_end_date = artist_df[['End Date', "source"]].drop_duplicates()
    source_end_date_latest = pd.DataFrame(source_end_date.groupby("source")["End Date"].max())
    source_end_date_latest = source_end_date_latest.reset_index()
    
    rank_max_end_date = artist_df[['End Date', 'track_name', "rank"]].drop_duplicates()

    min_rank = rank_max_end_date["rank"].min()
    max_rank = rank_max_end_date["rank"].max()

    min_rank_song_dt = rank_max_end_date[rank_max_end_date["rank"] == min_rank]
    max_rank_song_dt = rank_max_end_date[rank_max_end_date["rank"] == max_rank]

    min_rank_song_unique_dt = min_rank_song_dt[["End Date", "track_name"]].drop_duplicates()
    max_rank_song_unique_dt = max_rank_song_dt[["End Date", "track_name"]].drop_duplicates()

    source_end_date_latest_ls = source_end_date_latest.values.tolist()

    min_rank_song_unique_dt_ls = min_rank_song_unique_dt.values.tolist()
    max_rank_song_unique_dt_ls = max_rank_song_unique_dt.values.tolist()
    
    artist_inner_json = {}
    
    artist_inner_json["num_of_streams"] = str(num_of_streams)
    artist_inner_json["min_rank"] = str(min_rank)
    artist_inner_json["max_rank"] = str(max_rank)
    artist_inner_json["min_date"] = min_date
    artist_inner_json["max_date"] = max_date
    artist_inner_json["num_unique_songs"] = str(num_unique_songs)

    artist_inner_json["unique_songs"] = unique_songs

    artist_inner_json["source_end_date_latest_ls"] = dict(source_end_date_latest_ls)
    artist_inner_json["min_rank_song_unique_dt_ls"] = dict(min_rank_song_unique_dt_ls)
    artist_inner_json["max_rank_song_unique_dt_ls"] = dict(max_rank_song_unique_dt_ls)
    
    artist_json = {}
    artist_json[artist_name] = artist_inner_json
    
    return artist_json

In [4]:
unique_artists = all_files_za["main_artist"].unique().tolist()
all_artists_jsons = {}
    
for artist_name in unique_artists:  
    artist_json = getanartistsjson(all_files_za, artist_name)
    
    all_artists_jsons[artist_name] = artist_json
    # Serializing json
    json_object = json.dumps(artist_json, indent=4)
    
    new_artist_name = artist_name.translate(str.maketrans('', '', string.punctuation))

#     Writing to sample.json
#     with open("za_artists/" + new_artist_name +".json", "w") as outfile:
#         outfile.write(json_object)

### MAKE A SCRIPT for the voice over

In [5]:
max_date = all_files_za["End Date"].max()

In [6]:
max_date

'2022-10-06'

In [7]:
all_files_za_max_date = all_files_za[all_files_za["End Date"] == max_date]

In [8]:
unique_artists_latest = all_files_za_max_date["main_artist"].unique().tolist()

In [9]:
# unique_artists_latest

In [10]:
artists_of_interest = ['Kabza De Small', 'Blxckie', 'Busta 929', 'Mellow & Sleazy', 'Senior Oat', 'Wanitwa Mos', 'Mr JazziQ', 'Musa Keys', "Mas Musiq", "Deep London"]

In [11]:
# artists_of_interest = ["Mas Musiq", "Deep London"]

In [12]:
for anartist in artists_of_interest:
    voiceover_str = ""
    jsonartist = all_artists_jsons[anartist]
    
    thename = "Let us take a look at " + anartist + "'s journey on the Spotify Top 200 Weekly South Africa chart. \n "
    songs_streams = "He has had a total of " + jsonartist[anartist]["num_unique_songs"] + " songs with approximately " + jsonartist[anartist]["num_of_streams"] + " million streams. \n"
    
    record_labels = ""
    if(len(jsonartist[anartist]["source_end_date_latest_ls"].values()) == 1):
        record_labels = "As of " + max_date + " , he is with " + list(jsonartist[anartist]["source_end_date_latest_ls"].keys())[0]
    else:
        
        record_labels = "As of " + max_date + ", he has released music with " + ' .... '.join(list(jsonartist[anartist]["source_end_date_latest_ls"].keys()))
        
    appearance_date = "He first appeared on the chart on " + jsonartist[anartist]["min_date"] + "." + " His most recent appearance has been on " + jsonartist[anartist]["max_date"]
    
    min_rank_songs = list(set(list(jsonartist[anartist]["min_rank_song_unique_dt_ls"].values())))
    min_rank_songs_not_unique = (list(jsonartist[anartist]["min_rank_song_unique_dt_ls"].values()))
    
    max_rank_songs = list(set(list(jsonartist[anartist]["max_rank_song_unique_dt_ls"].values())))
    max_rank_songs_not_unique = list(jsonartist[anartist]["max_rank_song_unique_dt_ls"].values())
    
    min_rank_songs_dts_not_unique = list(jsonartist[anartist]["min_rank_song_unique_dt_ls"].keys())
    max_rank_songs_dts_not_unique = list(jsonartist[anartist]["max_rank_song_unique_dt_ls"].keys())
    
    positions_on_chart = "His highest position on the chart is " + jsonartist[anartist]["min_rank"] + " for the song "
    if(len(min_rank_songs) == 1):
        positions_on_chart +=  min_rank_songs[0] + " which was at this position on " + min_rank_songs_dts_not_unique[0]
    else:
        for the_dt in min_rank_songs_dts_not_unique:
            positions_on_chart += jsonartist[anartist]["min_rank_song_unique_dt_ls"][the_dt] + " which was at this position on " + the_dt + " "
    
    positions_on_chart_lowest = "His lowest position on the chart is " + jsonartist[anartist]["max_rank"] + " for the song "
    if(len(max_rank_songs) == 1):
        positions_on_chart_lowest +=  max_rank_songs[0] + " which was at this position on " + max_rank_songs_dts_not_unique[0]
    else:
        for the_dt in max_rank_songs_dts_not_unique:
            positions_on_chart_lowest += jsonartist[anartist]["max_rank_song_unique_dt_ls"][the_dt] + " which was at this position on " + the_dt + " "
            
    songs_on_chart = "The songs of his that have appeared on the chart are " + " ... ".join(jsonartist[anartist]["unique_songs"])
    
    voiceover_str = thename + songs_streams + record_labels + "\n"+ appearance_date + "\n" + positions_on_chart + "\n" + positions_on_chart_lowest + "\n" + songs_on_chart
    
    voiceover_str_json = {}
    voiceover_str_json[anartist] = voiceover_str
    # Serializing json
    json_object = json.dumps(voiceover_str_json, indent=4)
    
    new_artist_name = anartist.translate(str.maketrans('', '', string.punctuation))

#     Writing to sample.json
    print("za_artists/" + new_artist_name +"script.json")
    with open("za_artists/" + new_artist_name +"script.json", "w") as outfile:
        outfile.write(json_object)

za_artists/Kabza De Smallscript.json
za_artists/Blxckiescript.json
za_artists/Busta 929script.json
za_artists/Mellow  Sleazyscript.json
za_artists/Senior Oatscript.json
za_artists/Wanitwa Mosscript.json
za_artists/Mr JazziQscript.json
za_artists/Musa Keysscript.json
za_artists/Mas Musiqscript.json
za_artists/Deep Londonscript.json


In [13]:
# Blxckie's journey on the Spotify Top 200 South Africa chart.

# He has had a total of 21 songs with approximately 4 million streams. 

# As of 04 August 2022, he is with M4 Entertainment.

# He first appeared on the chart on 20 May 2021. His most recent appearance has been on 04 August 2022.

# His highest position on the chart is 3, for the song sneaky, which was at this position on 28 July 2022. 

# His lowest position was 199 with umoya on 9 June 2022. 

# The songs of his that have appeared on the chart are 